### Sociedades ligadas a personas naturales como fuente de grupos tributarios

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [2]:

spark = SparkSession.builder \
  .appName("Test")  \
  .config("spark.yarn.access.hadoopFileSystems","abfs://data@datalakesii.dfs.core.windows.net/DatoOrigen/lr-629/UtilBajo/intermedia/familiaridad") \
  .config("spark.executor.memory", "24g") \
  .config("spark.driver.memory", "12g")\
  .config("spark.executor.cores", "12") \
  .config("spark.executor.instances", "24") \
  .config("spark.driver.maxResultSize", "12g") \
  .getOrCreate()

24/04/16 03:41:13 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/04/16 03:41:13 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
Setting spark.hadoop.yarn.resourcemanager.principal to hvega.externo
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/16 03:41:14 WARN SparkConf: The configuration key 'spark.yarn.access.hadoopFileSystems' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.access.hadoopFileSystems' instead.
24/04/1

### Carga de relaciones societarias(depurada)

Se utilizan los datos de la malla de socios depurada.

In [3]:
df = spark.read.options(header=True, inferSchema=True, delimiter=",").csv("/home/cdsw/data/processed/malla_societaria_procesada/sociedades_participacion_capital_nozero.csv")
df.createOrReplaceTempView("sociedad")

24/04/16 03:42:05 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:42:20 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:42:35 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:42:50 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:43:05 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/04/16 03:43:20 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

### Tablas temporales previo a la ejecucion

In [4]:
spark.sql("select RUT_SOCIEDAD, RUT_SOCIO, PORCENTAJE_CAPITAL from sociedad order by RUT_SOCIEDAD asc").createOrReplaceTempView("sociedad")
spark.sql("select RUT_SOCIEDAD as RUT_SOCIEDAD_AUX ,RUT_SOCIO as RUT_SOCIO_AUX, PORCENTAJE_CAPITAL as PORCENTAJE_CAPITAL_AUX from sociedad order by RUT_SOCIEDAD asc").createOrReplaceTempView("aux")


### Calculo de arbol de socios naturales

El calculo se realiza con tablas auxiliares en 15 iteraciones. En cada iteracion se hace un join donde se van encadenando los socios por la malla societaria. Tamien se recalcula el porcentaje de participacion en relacion a la sociedad inicial.

In [5]:
for a in range (0,15):
    spark.sql("select * from sociedad left join aux on sociedad.RUT_SOCIO=aux.RUT_SOCIEDAD_AUX ").createOrReplaceTempView("sociedad")
    spark.sql("select * from sociedad order by RUT_SOCIEDAD_AUX desc").createOrReplaceTempView("sociedad")
    spark.sql("select RUT_SOCIEDAD, CASE WHEN RUT_SOCIEDAD_AUX is null then RUT_SOCIO else RUT_SOCIO_AUX END AS RUT_SOCIO, CASE when PORCENTAJE_CAPITAL_AUX is null then PORCENTAJE_CAPITAL else PORCENTAJE_CAPITAL_AUX*PORCENTAJE_CAPITAL/100  end as PORCENTAJE_CAPITAL from sociedad order by RUT_SOCIEDAD_AUX desc").createOrReplaceTempView("sociedad")
spark.sql("select * from sociedad order by RUT_SOCIEDAD asc").show()

+--------------------+--------------------+------------------+
|        RUT_SOCIEDAD|           RUT_SOCIO|PORCENTAJE_CAPITAL|
+--------------------+--------------------+------------------+
|+++JjoAr1RQa1kY7r...|S2zsiazT64Jiu/sSE...|             85.71|
|+++JjoAr1RQa1kY7r...|xO5P1s+Yeug4wKtb8...|             14.29|
|+++P6tU1TAUNG0SZv...|8ejqjUOvnKKfMZSzj...|             100.0|
|+++UeiptXaAQD74N1...|QemGi6c6gkD3rdstn...|              28.0|
|+++UeiptXaAQD74N1...|5jgmhcvBN2sVp7Gh5...|              72.0|
|+++VKgYcn1igYZjkT...|XRkbvgfaa5MRVblDF...|             100.0|
|+++VWXcqX/471v55m...|d/Gh1G45HEItM+byD...|              99.0|
|+++VWXcqX/471v55m...|t9AEh7tBjjp3FKWzm...|               1.0|
|+++ZYfC4XA28yQEQd...|Rf14vs+327ZlNsQvX...|             100.0|
|+++rmzjqUcW56fKfV...|VULdXIkUAxKz8BFIA...|              52.5|
|+++rmzjqUcW56fKfV...|/SogFhjTgxloEfGYI...|              47.5|
|++/BNOl5Jzp3/53dg...|ymLFBcIElT/XLhLlT...|              80.0|
|++/BNOl5Jzp3/53dg...|vkzKrESk+2mIrPN2k...|            

### Cruce con persona naturales y sociedades 

Obtendremos un output que nos permita establecer las sociedades relacionadas con cada uno de las personas naturales. Utilizaremos personas naturales que tengan cualquier porcentaje de participacion.

In [6]:
oscuridad=spark.sql("select * from libsdf.jab_materia_inom")
oscuridad.createOrReplaceTempView("oscuridad")
spark.sql("select RUT_SOCIEDAD, CONT_RUT, PORCENTAJE_CAPITAL from sociedad left join oscuridad on sociedad.RUT_SOCIO=oscuridad.CONT_RUT ").createOrReplaceTempView("socios_final")

24/04/16 03:52:18 WARN HiveClientImpl: Detected HiveConf hive.execution.engine is 'tez' and will be reset to 'mr' to disable useless hive logic
Hive Session ID = 22bb1faf-ec9e-4e62-8e20-765f0082ca34


Seleccionamos un porcentaje de capital mayor o igual a 0.5

In [7]:
sociedades_por_socio=spark.sql("select CONT_RUT as SOCIO_NATURAL, RUT_SOCIEDAD as SOCIEDAD_RELACIONADA,PORCENTAJE_CAPITAL from socios_final where CONT_RUT is not null and PORCENTAJE_CAPITAL>=0.5 order by RUT_SOCIEDAD DESC, PORCENTAJE_CAPITAL DESC")
sociedades_por_socio.show()
sociedades_por_socio.createOrReplaceTempView("socios_final")

+--------------------+--------------------+------------------+
|       SOCIO_NATURAL|SOCIEDAD_RELACIONADA|PORCENTAJE_CAPITAL|
+--------------------+--------------------+------------------+
|FktXvjNR5VXMJyG94...|zzzo7RUe8Wp/sBVZr...|              34.0|
|jSqaZci29rcIvTGYe...|zzzo7RUe8Wp/sBVZr...|              33.0|
|0BKm1Zv6LjcGstgdi...|zzzo7RUe8Wp/sBVZr...|              33.0|
|mGEnsfJQHpK0QrVBS...|zzzEfsBUrus3+Lpvp...|              50.0|
|q6tiscbhQ8ymF2ceM...|zzzEfsBUrus3+Lpvp...|              50.0|
|GPNxlTnbv8rq8xfZ7...|zzz9RqdyHZm0pUPEL...|             100.0|
|8CQvqjzxuyH3dbm5u...|zzz0aRgQrVHwDX+Ay...|              35.0|
|tRDN4t/AbFAPnkgrN...|zzz0aRgQrVHwDX+Ay...|              35.0|
|OUU34vltO18lw5auU...|zzz0aRgQrVHwDX+Ay...|              30.0|
|XggON0ZFlbniTNl8b...|zzykN+b2qc7ZoikA9...|              90.0|
|vjqqO+1+CetBiMMsb...|zzykN+b2qc7ZoikA9...|              10.0|
|u3d7ye5Mdt0h3gaEg...|zzydn/q5KUdW0Pcyx...|             100.0|
|oBwy7ODmDOAi3jGMi...|zzycSmylbx7abTn6L...|            

### Union con data de contaminados y caracteristicas

Ahora se seleccionan la persona natural asociada junto a cada una de las sociedades con su respectivo porcentaje de participacion y sus caracteristicas. Posteriormente guardamos el archivo.

In [11]:
spark.read.parquet("abfs://data@datalakesii.dfs.core.windows.net/DatosOrigen/lr-629/propuesta_f29_comunidades/data_contaminados_with_labels").createOrReplaceTempView("contaminados")
spark.sql('select * from contaminados').show()

+--------------------+--------------------+--------------+---------+--------------------+------------+--------+--------------+
|            cont_rut|               score|total_pago_f29| IVA_neto|     unidad_regional|n_documentos|lifetime|alerta_inicial|
+--------------------+--------------------+--------------+---------+--------------------+------------+--------+--------------+
|++EkqYV3XeaNiAsdf...| 0.05051744777816275|          null|    73844|8VA UNIDAD REGION...|        null|    null|             0|
|++dXZpaMXJgToW96Q...| 0.03939153827704531|           0.0|  1678385|        SANTIAGO SUR|        null|    null|             0|
|++olHY9+tFM+bjd3f...|0.048520650357892894|          null|    80378|2DA UNIDAD REGION...|        null|    null|             0|
|+/5sgPV/uzgox6653...|0.053509731789595616|          null|    42180|    SANTIAGO ORIENTE|        null|    null|             0|
|+/9/V7o5L54PLgASS...|  0.0542288535127804|           0.0|  8696000|   SANTIAGO PONIENTE|        null|    null|

In [12]:
spark.sql('select * from socios_final left join contaminados on socios_final.SOCIEDAD_RELACIONADA=contaminados.CONT_RUT where contaminados.CONT_RUT is not null').createOrReplaceTempView("aux")
spark.sql('select * from aux').show()

+--------------------+--------------------+------------------+--------------------+-------------------+--------------+----------+--------------------+------------+--------+--------------+
|       SOCIO_NATURAL|SOCIEDAD_RELACIONADA|PORCENTAJE_CAPITAL|            cont_rut|              score|total_pago_f29|  IVA_neto|     unidad_regional|n_documentos|lifetime|alerta_inicial|
+--------------------+--------------------+------------------+--------------------+-------------------+--------------+----------+--------------------+------------+--------+--------------+
|G1NPZeFPbEvK63iXK...|++0TZZ4dLrh7Mcrrq...|             90.91|++0TZZ4dLrh7Mcrrq...|0.48523924596416235|   2.7701443E7|-117348357|7MA UNIDAD REGION...|         193|    2520|             0|
|HuScCDbt9021COD28...|++0TZZ4dLrh7Mcrrq...|              9.09|++0TZZ4dLrh7Mcrrq...|0.48523924596416235|   2.7701443E7|-117348357|7MA UNIDAD REGION...|         193|    2520|             0|
|w2OxfuiPP3mbDXj0h...|++Q4xcstVE8gsYp69...|             100.

+--------------------+--------------------+------------------+--------------------+--------------+-----------+--------------------+------------+--------+--------------+
|       SOCIO_NATURAL|SOCIEDAD_RELACIONADA|PORCENTAJE_CAPITAL|               score|total_pago_f29|   IVA_neto|     unidad_regional|n_documentos|lifetime|alerta_inicial|
+--------------------+--------------------+------------------+--------------------+--------------+-----------+--------------------+------------+--------+--------------+
|+++Dv3VdgQyVqRTgl...|jIjACgvcsprjqLeup...|              24.0|0.052408941789938795|   9.6336447E7|   -5282491|                null|           4|     530|             0|
|+++YLFFod6As3+EIq...|XdNCugfCv/JNxkjsW...|             16.67| 0.06730458372836294|   1.7331782E7|  -12565236|6TA UNIDAD REGION...|          10|    3451|             0|
|+++cWi/r31w33XdNY...|dqMQkYCsU9PCaLwUT...|             100.0|  0.0461122114462416|        9500.0|    -955065|                null|           6|     633|  

In [15]:
aux=spark.sql('select SOCIO_NATURAL,SOCIEDAD_RELACIONADA,PORCENTAJE_CAPITAL,score,total_pago_f29,IVA_neto,unidad_regional,n_documentos,lifetime,alerta_inicial from aux order by SOCIO_NATURAL asc')
aux.show()

+--------------------+--------------------+------------------+--------------------+--------------+-----------+--------------------+------------+--------+--------------+
|       SOCIO_NATURAL|SOCIEDAD_RELACIONADA|PORCENTAJE_CAPITAL|               score|total_pago_f29|   IVA_neto|     unidad_regional|n_documentos|lifetime|alerta_inicial|
+--------------------+--------------------+------------------+--------------------+--------------+-----------+--------------------+------------+--------+--------------+
|+++Dv3VdgQyVqRTgl...|jIjACgvcsprjqLeup...|              24.0|0.052408941789938795|   9.6336447E7|   -5282491|                null|           4|     530|             0|
|+++YLFFod6As3+EIq...|XdNCugfCv/JNxkjsW...|             16.67| 0.06730458372836294|   1.7331782E7|  -12565236|6TA UNIDAD REGION...|          10|    3451|             0|
|+++cWi/r31w33XdNY...|dqMQkYCsU9PCaLwUT...|             100.0|  0.0461122114462416|        9500.0|    -955065|                null|           6|     633|  

Guardamos el archivo final con 

In [16]:
aux=aux.toPandas()
aux.to_csv('/home/cdsw/data/processed/comunidades_persona_natural/comunidades_natural_sociedades.csv', index=False)